In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from portfolio_optimization import PortfolioOptimization

In [3]:
import numpy as np
import pandas as pd
# from pandas_datareader import data

from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns

from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import plotting
from copy import deepcopy

import pandas_datareader as pdr

In [ ]:
blue_chips_names = np.array(
    ['GAZP', 'GEMC', 'YNDX', 'VKCO', 'POSI', 'FLOT', 'UNAC', 'NMTP', 'FESH', 'PIKK', 'AFLT',
     'DSKY', 'MVID', 'MGNT', 'OZON', 'POLY', 'ALRS', 'NLMK', 'PHOR', 'MTSS', 'SBER', 'TCSG', 
     'IRAO', 'NVTK', 'ROSN']
)
blue_chips_names.astype(str)

In [ ]:
blue_chips_names = np.array(
    [
        'POLY', 'FIVE', 'GAZP', 'GMKN', 'LKOH', 'MGNT',	'MTSS', 'NLMK', 'NVTK', 'PLZL', 'ROSN', 
        'SBER', 'SNGS', 'TATN', 'YNDX'
    ]
)

In [ ]:
df_assets = small_df.drop(['GEMC', 'VKCO', 'POSI', 'FLOT', 'TCSG', 'OZON'], axis=1)